In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import catboost as cb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
%%capture
d = pd.read_csv('train_data.csv', error_bad_lines=False, header=0)

In [3]:
d.head()

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit
0,0,J06.0,Острый ларингофарингит,29,2,NaN,NaN,NaN,На жевательной поверхности 2.6- кариозная поло...,1
1,1,N76.1,Подострый и хронический вагинит,45,2,NaN,NaN,считает себя больной на протяжении многих лет....,NaN,1
2,2,L23.9,"Аллергический контактный дерматит, причина не ...",63,2,NaN,NaN,"Болен около 2-х дней, когда появились описанны...",NaN,1
3,3,N48.1,Баланопостит,43,2,NaN,не отягощен,NaN,NaN,1
4,4,Z00.0,Общий медицинский осмотр,29,1,NaN,NaN,провела полное обследование для оперативного л...,NaN,1


In [4]:
d.shape

(283086, 10)

In [5]:
df = pd.read_csv('mkb10.csv', error_bad_lines=False, header=0)

In [6]:
df.head()

,Код,Название,Родителя
0,A00-B99,Некоторые инфекционные и паразитарные болезни,NaN
1,A00-A09,Кишечные инфекции,A00-B99
2,A00,Холера,A00-A09
3,A00.0,"Холера, вызванная холерным вибрионом 01, биова...",A00
4,A00.1,"Холера, вызванная холерным вибрионом 01, биова...",A00


In [7]:
def find_class_of_disease(s):
    parent = df[df.Код == s].iloc[0, 2]
    l = [s, df[df.Код == s].iloc[0, 1]]
    while not pd.isnull(parent):
        s = parent
        l.append(s)
        l.append(df[df.Код == s].iloc[0, 1])
        parent = df[df.Код == s].iloc[0, 2]
    if len(l) == 6:
        l.insert(0, l[1])
        l.insert(0, l[1])
    if len(l) == 8:
        l.insert(len(l) - 2, l[-2])
        l.insert(len(l) - 2, l[-1])
    return l
find_class_of_disease('A00')

['A00',
 'Холера',
 'A00',
 'Холера',
 'A00-A09',
 'Кишечные инфекции',
 'A00-B99',
 'Некоторые инфекционные и паразитарные болезни',
 'A00-B99',
 'Некоторые инфекционные и паразитарные болезни']

In [8]:
d.iloc[13159, 1] =  'A04'
d.iloc[15896, 1] = 'T76.8'
d.iloc[24848, 1] = 'T76.8'
d.iloc[25652, 1] = 'A03.8'
d.iloc[47761, 1] = 'A04.9'
d.iloc[55274, 1] = 'A04.9'
d.iloc[55769, 1] = 'A04.9'
d.iloc[60456, 1] = 'A07.1'
d.iloc[82176, 1] = 'A07.1'
d.iloc[87098, 1] = 'A03.1'
d.iloc[88119, 1] = 'A04.9'
d.iloc[93778, 1] = 'T76.8'
d.iloc[103545, 1] = 'A08.4'
d.iloc[109548, 1] = 'A08.4'
d.iloc[110483, 1] = 'A04.9'
d.iloc[115065, 1] = 'T76.8'
d.iloc[116228, 1] = 'A15.6'
d.iloc[140154, 1] = 'T76.8'  #Другие уточненные воспалительные болезни влага
d.iloc[164281, 1] = 'A04.9'  #Бактериальная кишечная инфекция неуточненна
d.iloc[181572, 1] = 'A07.1' #Жиардиаз
d.iloc[190383, 1] = 'T76.8'
d.iloc[227788, 1] = 'A04.9'
d.iloc[244603, 1] = 'A15.8'
d.iloc[251455, 1] = 'A04.9'
d.iloc[253312, 1] = 'A07.1'
d.iloc[257096, 1] = 'A05.9'
d.iloc[274090, 1] = 'A07.1'


In [ ]:
from tqdm import tqdm

l = 0
new_data=[]
for i in tqdm(range(d.shape[0])):
    l = find_class_of_disease(d.iloc[i,1])
    new_data.append(l)

In [2]:
new_data = pd.read_csv('test.csv', error_bad_lines=False, header=0)

In [4]:
transpose_new_data = list(map(list, zip(*new_data)))

In [ ]:
len(transpose_new_data[0])

In [ ]:
k = pd.DataFrame(transpose_new_data)
k.to_csv('new_data_mrp.csv', index=False, header=False)